# Column to Tabular Schema
This notebook is an example of how purviewcli can be used to:  

  1. Create a batch of entities (typeName: column).
  2. Programatically associate those entities to a tabular schema by establishing a "relationship".

## Pre-requisites

In [ ]:
# Initialise Helper Method
import json
def getJSON(raw_output):
    output = ''.join(raw_output)
    json_obj = json.loads(output)
    return json_obj

In [ ]:
# Install/Upgrade purviewcli package
%pip install --upgrade pvw-cli

In [ ]:
# Set Environment Variables
%env PURVIEW_ACCOUNT_NAME=YOUR_PURVIEW_ACCOUNT
%env AZURE_CLIENT_ID=YOUR_CLIENT_ID
%env AZURE_TENANT_ID=YOUR_TENANT_ID
%env AZURE_CLIENT_SECRET=YOUR_CLIENT_SECRET

## Script

In [ ]:
# Step 1: Create a batch of entities (typeName: column)
data = !pvw entity createBulk --payloadFile "../json/entity/column_entities.json"
response = getJSON(data)
print(response)

In [ ]:
# Step 2: Loop through entities and establish a relationship to the parent tabular schema (relationship type: tabular_schema_columns)
counter = 0
for entity in response['mutatedEntities']['CREATE']:
    counter += 1
    guid = entity['guid']
    qualifiedName = entity['attributes']['qualifiedName']
    relationship_payload = {
        "end1": {
            "guid": "540e4f2f-66ac-49b0-b5c1-98f6f6f60000",
            "typeName": "tabular_schema",
            "uniqueAttributes": {
                "qualifiedName": "https://adls8951.dfs.core.windows.net/other/lots_of_cols.csv#__tabular_schema"
            }
        },
        "end2": {
            "guid": f"{guid}",
            "typeName": "column",
            "uniqueAttributes": {
                "qualifiedName": f"{qualifiedName}"
            }
        },
        "status": "ACTIVE",
        "typeName": "tabular_schema_columns"
    }
    filename = f'rel_{counter}.json'
    with open(filename,'w') as f:
        json.dump(relationship_payload, f)
    !pvw relationship create --payloadFile {filename}